In [1]:
import json
from collections import defaultdict
import pandas as pd

In [2]:
df = pd.read_csv('candidate-transfer.csv')
df['from'], df['to'], df['hash'] = df['from'].apply(lambda x: x[2:]), df['to'].apply(lambda x: x[2:]), df['hash'].apply(lambda x: x[2:])
df['block_time'] = pd.to_datetime(df['block_time'])
df = df.loc[df['block_time'] > pd.to_datetime('2021-01-01 15:47:18+00:00')]

candidate_address = set(df['to'].unique())
print("Candidate transactions: ", df.shape[0])
print("Candidate addresses: ", len(candidate_address))

Candidate transactions:  89285
Candidate addresses:  4777


In [3]:
# Selected the transactions whose "from" and "to" both interactived with Hop USDC Bridge
df = df[df['from'].isin(candidate_address)]
df = df.sort_values('block_time').drop_duplicates(('from', 'to'), keep='last')

# The output [781cdc81cc8a315433bdc1c581a649c4c0d61ee7    32] means the address 0x781cdc81cc8a315433bdc1c581a649c4c0d61ee7 has transfered to his/her 32 child-addresses, and these 33 addresses all have interactived with Hop USDC Bridge!
df['from'].value_counts()

781cdc81cc8a315433bdc1c581a649c4c0d61ee7    32
a822390152cb85db84613fde7676e0b36ebfb334    19
c67f6fbd90806efcac2b7afbafb038afe4df5a0c    10
9ab119f2017358dd33894fef1d20f870e3f70850     9
6649a5d4ae7ac73f2f6b6baf3b722d8dcb281e33     9
                                            ..
318fb4d4de8def84b5b836ee57c0dbf87cf50f7d     1
5f67b369e5155edc4270f1e2615b70dac41e93ca     1
119976a1cfe7c109ef52ce3f74707ab574a526a7     1
ba2dc20329b21370b91e1306b1c0b4c0ff9adbd5     1
9633d229b711dcbf06cdd8739238765e0f3135b0     1
Name: from, Length: 726, dtype: int64

In [4]:
# Save the results to the json file.
save_file = defaultdict(lambda: [0, []])

for _, (from_address, to_address, _, tx_hash) in df.iterrows():
    save_file[from_address][0] += 1
    save_file[from_address][1].append((to_address, tx_hash))
    
save_file = dict(sorted(list(save_file.items()), key=lambda x: x[1][0], reverse=True))
json.dump(save_file, open('report_address.json', 'w'), indent=2)